In [1]:
import docx2txt as docx
text = docx.process("./Resume_2.docx")

In [2]:
text = text.replace("\n"," ")
text
# text = text.replace("\t"," ")

'ABC. ERR   Mob- 576857687                     \t\t\t\t                                ACADEMIC QUALIFICATIONS  Examination  Year  Board/University  %/CGPA  MBA  2010  ABC  8.0/9.0  B.Tech (ABC)  2008  N. ABC  9.98/10  CLASS XII  2003  ABC  33.60  CLASS X  2001  ABC  94.50  WORK EXPERIENCE  \tSales & Strategy                                         ABC -Up                                                                   Oct’14 – Feb’15  \tABC  \t\tIdeated product design of Stewot- app for restaurants end-to-end operations management  \t\tDeveloped database of 300+ restaurants for sales through cold calling and demonstrations  \tABC Manager                                              ABC                                                                                July’08-May’12  Process        In-charge   \t\tProcess In-charge for  Rodding  plant with 23.pa. Carbon anode production capacity  \t\tCertified Lean Six sigma Black belt   Lean 6-sigma Project  \t\tTeam member in applying 

In [4]:
#email
import re
remail = re.compile(r'[\w\.-]+@[\w\.-]+')
email = remail.search(text)
if email:
    email=email[0]
else:
    email = "no email found"
print(email)

no email found


In [5]:
#linkedin profile
s = text
rlink = re.compile(r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))""")
link = rlink.search(s)
if link:
    link=link[0]
else:
    link="no link found"
print (link)

no link found


In [6]:
r = re.compile(r"(([(]?[0-9]{1,3}[)]?)|([(]?[0-9]{4}[)]?))\s*[)]?[-\s\.]?[(]?[0-9]{1,3}[)]?([-\s\.]?[0-9]{3})([-\s\.]?[0-9]{3,4})")
phone = r.search(text)
if phone:
    phone=phone[0]
else:
    phone="Phone number not found"
print(phone)

576857687


In [7]:
import re
import nltk
from nltk.corpus import stopwords
import pprint
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

document = text[:int(len(text)/50)]


stoplist = ["phone","ph","email","linkedin","resume","cv","curriculum vitae","mob"]
stop = stopwords.words('english')
stop.extend(stoplist)

names = []
document = text[:int(len(text)/50)]
document = ' '.join([i for i in document.split() if i not in stop])
sentences = nltk.sent_tokenize(document)
sentences = [nltk.word_tokenize(sent) for sent in sentences]
sentences = [nltk.pos_tag(sent) for sent in sentences]

lines = sentences

indianNames = open("names.txt", "r").read().lower()
indianNames = set(indianNames.split())

otherNameHits = []
nameHits = []
name = None
infoDict={}

grammar = r'NAME: {<NN.*><NN.*><NN.*>*}'
chunkParser = nltk.RegexpParser(grammar)
all_chunked_tokens = []
for tagged_tokens in lines:
    if len(tagged_tokens) == 0: continue
    chunked_tokens = chunkParser.parse(tagged_tokens)
    all_chunked_tokens.append(chunked_tokens)
    for subtree in chunked_tokens.subtrees():
        if subtree.label() == 'NAME':
            for ind, leaf in enumerate(subtree.leaves()):
                if leaf[0].lower() in indianNames and 'NN' in leaf[1]:
                    hit = " ".join([el[0] for el in subtree.leaves()[ind:ind+3]])
                    if re.compile(r'[\d,:]').search(hit): continue
                    nameHits.append(hit)
if len(nameHits) > 0:
    nameHits = [re.sub(r'[^a-zA-Z \-]', '', el).strip() for el in nameHits] 
    name = " ".join([el[0].upper()+el[1:].lower() for el in nameHits[0].split() if len(el)>0])

if name:
    infoDict['name'] = name
else:
    st = text[:int(len(text)/70)].replace("\t","").replace(link,"").replace(email,"").replace(phone,"").strip().lower()
    st = re.sub(r'[^\w]', ' ', st)
    st = ' '.join([i for i in st.split() if i not in stoplist])
    infoDict['name'] = "Name not found. It may be in string : '"+st+"'"

In [8]:
infoDict

{'name': "Name not found. It may be in string : 'abc err'"}